## **09: Tree-based Models**
- Instructor: [Jaeung Sim](https://jaeungs.github.io/) (University of Connecticut)
- Course: OPIM 5512 Data Science Using Python
- Date: April 3, 2025

**Objectives**
1. Implement and evaluate tree-based models for regression and classification problems.
1. Understand and implement random forest models.
1. Understand and implement gradient boosting and XGBoost.

**References**
* [Diamonds (Kaggle)](https://www.kaggle.com/datasets/shivam2503/diamonds)
* An Introduction to Statistical Learning with Applications in R (ISLR) ([free lectures on YouTube](https://www.youtube.com/@datascienceanalytics1826/))
* [Decision Trees (scikit-learn)](https://scikit-learn.org/stable/modules/tree.html)
* [Random Forest Regression in Python](https://www.geeksforgeeks.org/random-forest-regression-in-python/)
* [How to Solve Overfitting in Random Forest in Python Sklearn?](https://www.geeksforgeeks.org/how-to-solve-overfitting-in-random-forest-in-python-sklearn/?ref=rp)
* [Bagging vs Boosting in Machine Learning](https://www.geeksforgeeks.org/bagging-vs-boosting-in-machine-learning/)
* [Understanding the Bias-Variance Tradeoff](https://towardsdatascience.com/understanding-the-bias-variance-tradeoff-165e6942b229)
* [Understanding Out-of-Bag (OOB) Score: Random Forest Algorithm Evaluation](https://www.analyticsvidhya.com/blog/2020/12/out-of-bag-oob-score-in-the-random-forest-algorithm/)
* [StatQuest](https://statquest.org/)
* [How to Develop a Gradient Boosting Machine Ensemble in Python](https://machinelearningmastery.com/gradient-boosting-machine-ensemble-in-python/)
* [Using XGBoost in Python Tutorial](https://www.datacamp.com/tutorial/xgboost-in-python)

##### **Part 0. Basic Setup**

In [ ]:
# Import basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split

**Diamonds Dataset**

We will be working with the Diamonds dataset throughout the tutorial. It is built into the Seaborn library, or alternatively, you can also download it from [Kaggle](https://www.kaggle.com/datasets/shivam2503/diamonds). It has a nice combination of numeric and categorical features of almost 54,000 diamonds.

>**Variables**
* **price:** price in US dollars (\$326 - \$18,823)
* **carat:** weight of the diamond (0.2 - 5.01)
* **cut:** quality of the cut (Fair, Good, Very Good, Premium, Ideal)
* **color:** diamond colour, from J (worst) to D (best)
* **clarity:** a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))
* **x:** length in mm (0 - 10.74)
* **y:** width in mm (0 - 58.9)
* **z:** depth in mm (0 - 31.8)
* **depth:** total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43--79)
* **table:** width of top of diamond relative to widest point (43--95)

In [ ]:
# Load and explore the dataset
diamonds = sns.load_dataset("diamonds")
diamonds.head()

In [ ]:
# Summary Statistics (Numeric Variables)
diamonds.describe()

In [ ]:
# Summary Statistics (Categorical Variables)
diamonds.describe(exclude=np.number)

##### **Part 1. Decision Tree Models**

**Terminology for Trees**
* In keeping with the *tree* analogy, the regions divided by the model are called *terminal nodes* or *leaf nodes*.
* Decision trees are typically drawn *upside down*, in the sense that the leaves are at the bottom of the tree.
* The points along the tree where the predictor space is split are referred to as *internal nodes* or *decision nodes*.

<div>
<img src="https://365datascience.com/resources/blog/rr6cuudl59r-decision-trees-image1.png" width="700"/>
</div>

When people make a decision to accept a job offer...

<div>
<img src="https://365datascience.com/resources/blog/59utffqewug-decision-trees-image2.png" width="700"/>
</div>

**Pros and Cons**
* Tree-based methods are simple and useful for interpretation.
* However, they typically are not competitive with the best supervised learning approaches in terms of prediction accuracy.
* Therefore, we also discuss *bagging*, *random forests*, and *boosting*. These methods grow multiple trees which are then combined to yield a single consensus prediction.
* Combining a large number of trees can often result in dramatic improvements in prediction accuracy, at the expense of some loss interpretation.

**Tree-Based vs. Linear Models in Classification**

<div>
<img src="https://s3.amazonaws.com/media-p.slid.es/uploads/757574/images/6761463/8.7.png" width="700"/>
</div>

**1.1. Regression**

**Overview of the Tree-Building Process**
1. We divide the predictor space --- that is, the set of possible values for $X_1, X_2, ..., X_p$ --- into $J$ distinct and non-overlapping regions, $R_1, R_2, ..., R_J$.
1. For every observation that falls into the region $R_j$, we make the same prediction, which is simply the mean of the response values for the training observations in $R_j$.

**Objective and Approach**
* In theory, the regions could have any shape. However, we choose to divide the predictor space into high-dimensional rectangles, or *boxes*, for simplicity and for ease of interpretation of the resulting predictive model.
* The goal is to find boxes $R_1, ..., R_J$ that minimize the RSS, given by
> $\Sigma_{j=1}^J \Sigma_{i \in R_j} (y_i - \hat{y}_{R_j})^2$,
>
> where $\hat{y}_{R_j}$ is the mean response for the training observations within the *j*th box.
* Unfortunately, it is computationally infeasible to consider every possible partition of the feature space into *J* boxes.
* For this reason, we take a *top-down*, *greedy* approach that is known as recursive binary splitting.
* The approach is *top-down* because it begins at the top of the tree and then successively splits the predictor space; each split is indicated via two new branches further down on the tree.
* It is *greedy* because at each step of the tree building process, the *best* split is made at that particular step, rather than looking ahead and picking a split that will lead to a better tree in some future step.

**Details of the Process**
* We first select the predictor $X_j$ and the cutpoint $s$ such that splitting the predictor space into the regions $\{ X|X_j < s \}$ and $\{ X|X_j \ge s \}$ leads to the greatest possible reduction in RSS.
* Next, we repeat the process, looking for the best predictor and best cutpoint in order to split the data further so as to minimize the RSS within each of the resulting regions.
* However, this time, instead of splitting the entire predictor space, we split one of the two previously identified regions. We now have three regions.
* Again, we look to split one of these three regions further, so as to minimize the RSS. The process continues until a stopping criterion is reached; for instance, we may continue until no region contains more than a certain number of observations.

In [ ]:
# Import libraries
from sklearn import tree

**Data Preparation**

In [ ]:
# Extract feature and target arrays
X, y = diamonds.drop('price', axis=1), diamonds[['price']]

In [ ]:
# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

In [ ]:
# Convert to Pandas category
for col in cats:
   X[col] = X[col].astype('category')

In [ ]:
# Create a copy of the original DataFrame (otherwise, your model doesn't work)
X2 = X.copy()

# Categorical columns
cut_dummies = pd.get_dummies(X2["cut"], prefix="cut")
color_dummies = pd.get_dummies(X2["color"], prefix="color")
clarity_dummies = pd.get_dummies(X2["clarity"], prefix="clarity")

# Concatenate these variables
X2 = pd.concat([X2, cut_dummies, color_dummies, clarity_dummies], axis=1)

# Drop categorical variables
X2 = X2.drop(["cut", "color", "clarity"], axis=1)
X2

In [ ]:
# Split the data
X2_train, X2_test, y_train, y_test = train_test_split(X2, y, random_state=10)

**Model Implementation**

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Create regressor object
reg1 = DecisionTreeRegressor(max_depth=2)
reg2 = DecisionTreeRegressor(max_depth=5)

# fit the regressor with x and y data
reg1.fit(X2_train, y_train)
reg2.fit(X2_train, y_train)

In [ ]:
# Predict
y_pred1 = reg1.predict(X2_test)
y_pred2 = reg2.predict(X2_test)

In [ ]:
# Plot the results
plt.figure()
plt.scatter(X2['carat'], y, s=20, edgecolor="black", c="darkorange", label="data")
plt.scatter(X2_test['carat'], y_pred1, color="cornflowerblue", label="max_depth=2", linewidth=1)
plt.scatter(X2_test['carat'], y_pred2, color="yellowgreen", label="max_depth=5", linewidth=1)
plt.xlabel("Carat")
plt.ylabel("Price")
plt.title("Decision Tree Regression")
plt.legend()
plt.show()

<div>
<img src="https://miro.medium.com/v2/resize:fit:640/format:webp/1*xwtSpR_zg7j7zusa4IDHNQ.png" width="500"/>
</div>

**Bias vs. Variance in Machine Learning**
* **Bias** is the difference between the average prediction of our model and the correct value which we are trying to predict. Model with high bias pays very little attention to the training data and oversimplifies the model. It always leads to high error on training and test data.
* **Variance** is the variability of model prediction for a given data point or a value which tells us spread of our data. Model with high variance pays a lot of attention to training data and does not generalize on the data which it hasn’t seen before. As a result, such models perform very well on training data but has high error rates on test data.

<div>
<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*9hPX9pAO3jqLrzt0IE3JzA.png" width="700"/>
</div>

**Complexity and Trade-Off**
* A complex tree may *overfit* the data, leading to poor test set performance.
* A smaller tree with fewer splits (that is, fewer regions $R_1, ..., R_J$) might lead to lower variance and better interpretation at the cost of a little bias.
* One possible alternative is to grow the tree only so long as the decrease in the RSS due to each split exceeds some (high) threshold.
* This strategy will result in a smaller tress, but is too *short-sighted*: a seemingly worthless split early on in the tree might be followed by a very good split --- that is, a split that leads to a large reduction in RSS later on.

**Pruning a Tree**
* A better strategy is to grow a very large tree $T_0$, and then *prune* it back in order to obtain a *subtree*.
* *Cost complexity pruning* --- also known as *weakest link pruning* --- is used to do this.
* We consider a sequence of trees indexed by a nonnegative tuning parameter $\alpha$. For each value of $\alpha$ there corresponds a subtree $T \subset T_0$ such that
>$\Sigma_{m=1}^{|T|} \Sigma_{x_i \in R_m} (y_i - \hat{y}_{R_m})^2 + \alpha |T|$,
>
>is as small as possible. Here |T| indicates the number of terminal nodes of the tree $T$, $R_m$ is the rectangle (i.e., the subset of predictor space) corresponding to the *m*th terminal node, and $\hat{y}_{R_m}$ is the mean of the training observations in $R_m$.

**Choosing the Best Subtree**
* The tuning parameter $\alpha$ controls a trade-off between the subtree's complexity and its fit to the training data.
* We select an optimal value $\hat{\alpha}$ using cross-validation.
* We then return to the full data set and obtain the subtree corresponding to $\hat{\alpha}$.

**1.2. Classification**

**Classification Trees**
* Very similar to a regression tree, except that it is used to predict a qualitative response rather than a quantitative one.
* For a classification tree, we predict that each observation belongs to the *most commonly occurring class* of training observations in the region to which it belongs.

**Details of Classification Trees**
* Just as in the regression setting, we use recursive binary splitting to grow a classification tree.
* In the classification setting, RSS cannot be used as a criterion for making the binary splits.
* A natural alternative to RSS is the *classification error rate*. This is simply the fraction of the training observations in that region that do not belong to the most common class:
>$E=1-max_{k}(\hat{p}_{mk})$,
>
>where $\hat{p}_{mk}$ represents the proportion of training observations in the *m*th region that are from the *k*th class.
* However, classification error is not sufficiently sensitive for tree-growing, and in practice two other measures are preferable.

**Gini Index**
* The *Gini index* is defined by
>$G=\Sigma_{k=1}^K \hat{p}_{mk} (1 - \hat{p}_{mk})$,
>
>a measure of total variance across the *K* classes. The Gini index takes on a small value if all of the $\hat{p}_{mk}$'s are close to zero or one.
* For this reason, the Gini index is referred to as a measure of node *purity* --- a small value indicates that a node contains observations from a single class predominantly.
* An alternative to the Gini index is *cross-entropy*, given by
>$D=-\Sigma_{k=1}^K \hat{p}_{mk} log (\hat{p}_{mk})$.
* It turns out that the Gini index and the cross-entropy are numerically very similar.

**Data Preparation**

In [ ]:
# Extract feature and target arrays
X, y = diamonds.drop("cut", axis=1), diamonds[['cut']]

In [ ]:
# See how 'y' looks
y

In [ ]:
# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

In [ ]:
# Convert to pd.Categorical
for col in cats:
   X[col] = X[col].astype('category')

In [ ]:
# Create a copy of the original DataFrame
X2 = X.copy()

# Categorical columns
color_dummies = pd.get_dummies(X2["color"], prefix="color")
clarity_dummies = pd.get_dummies(X2["clarity"], prefix="clarity")

# Concatenate these variables
X2 = pd.concat([X2, color_dummies, clarity_dummies], axis=1)

# Drop categorical variables
X2 = X2.drop(["color", "clarity"], axis=1)
X2

In [ ]:
# Split the data
X2_train, X2_test, y_train, y_test = train_test_split(X2, y, random_state=10)

**Model Implementation**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Estimate the model
clf = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=10)
clf.fit(X2_train, y_train)

In [ ]:
from sklearn import tree

# Visualize the estimated tree
tree.plot_tree(clf)

In [ ]:
# More Comprehensible
fig, axes = plt.subplots(nrows = 1, ncols = 1, dpi=300)
tree.plot_tree(clf, filled = True);

In [ ]:
# Total impurity vs effective alpha for train set
path = clf.cost_complexity_pruning_path(X2_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

# Plot the Figure
fig, ax = plt.subplots()
ax.plot(ccp_alphas[:-1], impurities[:-1], marker="o", drawstyle="steps-post")
ax.set_xlabel("effective alpha")
ax.set_ylabel("total impurity of leaves")
ax.set_title("Total Impurity vs effective alpha for training set")

In [ ]:
# Train a tree using the effective alphas
clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=10, ccp_alpha=ccp_alpha)
    clf.fit(X2_train, y_train)
    clfs.append(clf)
print(
    "Number of nodes in the last tree is: {} with ccp_alpha: {}".format(
        clfs[-1].tree_.node_count, ccp_alphas[-1]
    )
)

In [ ]:
# Number of Nodes vs. Alpha / Depth vs. Alpha
clfs = clfs[:-1]
ccp_alphas = ccp_alphas[:-1]

node_counts = [clf.tree_.node_count for clf in clfs]
depth = [clf.tree_.max_depth for clf in clfs]
fig, ax = plt.subplots(2, 1)
ax[0].plot(ccp_alphas, node_counts, marker="o", drawstyle="steps-post")
ax[0].set_xlabel("alpha")
ax[0].set_ylabel("number of nodes")
ax[0].set_title("Number of nodes vs alpha")
ax[1].plot(ccp_alphas, depth, marker="o", drawstyle="steps-post")
ax[1].set_xlabel("alpha")
ax[1].set_ylabel("depth of tree")
ax[1].set_title("Depth vs alpha")
fig.tight_layout()

In [ ]:
# Accuracy vs. Alpha for Train and Test Sets
train_scores = [clf.score(X2_train, y_train) for clf in clfs]
test_scores = [clf.score(X2_test, y_test) for clf in clfs]

fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha for training and testing sets")
ax.plot(ccp_alphas, train_scores, marker="o", label="train", drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker="o", label="test", drawstyle="steps-post")
ax.legend()
plt.show()

##### **Part 2. Random Forest**

**Forest = A Set of Trees**

An ensemble learning method that combines the outputs of multiple decision trees to reach a single result

<div>
<img src="https://www.spotfire.com/content/dam/spotfire/images/graphics/inforgraphics/random-forest-diagram.svg" width="700"/>
</div>

**Bagging**
* *Bootstrap aggregation*, or *bagging*, is a general-purpose procedure for reducing the variance of a statistical learning method; we introduce it here because it is particularly useful and frequently used in the context of decision trees.
* Recall that given a set of $n$ independent observations $Z_1, ..., Z_n$, each with variance $\sigma^2$, the variance of the mean $\bar{Z}$ of the observations is given by $\sigma^2/n$.
* In other words, *averaging a set of observations reduces variance.* Of course, this is not practical because we generally do not have access to multiple training sets.
* Instead, we can bootstrap, by taking repeated samples from the (single) training dataset. In this approach, we generate $B$ different bootstrapped training data sets. We then train our method on the *b*th bootstrapped training set in order to get $\hat{f}^{*b}$, the prediction at a point $x$. We then average all the predictions to obtain
>$\hat{f}_{bag}(x) = \frac{1}{B} \Sigma_{b=1}^B \hat{f}^{*b}(x)$.
>
>This is called *bagging*.

**Out-of-Bag Error Estimation**
* It turns out that there is a very straightforward way to estimate the test error of a bagged model.
* Recall that the key to bagging is that trees are repeatedly fit to bootstrapped subsets of the observations. One can show that on average, each bagged tree makes use of around two-thirds of the observations
* The remaining one-third of the observations not used to fit a given bagged tree are referred to as the *out-of-bag* (OOB) observations.
* We can predict the response for the *i*th observation using each of the trees in which that observation was OOB. This will yield around $B/3$ predictions for the *i*th observation, which we average.

**Random Forests**
* *Random forests* provide an improvement over bagged trees by way of a small tweak that *decorrelates* the trees. This reduces the variance when we average the trees.
* As in bagging, we build decision trees on bootstrapped training samples.
* But when building these decision trees, each time a split in a tree is considered, *a random selection of m predictors* is chosen as split candidates from the full set of *p* parameters. The split is allowed to use only one of those *m* predictors.
* A fresh selection of *m* predictors is taken at each split, and typically, we choose $m \thickapprox \sqrt{p}$. That is, the number of predictors considered at each split is approximately equal to the square root of the total number of predictors.

In [ ]:
# import the regressor
from sklearn.ensemble import RandomForestRegressor

**Data Preparation**

In [ ]:
# Extract feature and target arrays
X, y = diamonds.drop('price', axis=1), diamonds[['price']]

In [ ]:
# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

In [ ]:
# Convert to Pandas category
for col in cats:
   X[col] = X[col].astype('category')

In [ ]:
# Check the variable types
X.dtypes

**Model Implementation**

In [ ]:
# Create a copy of the original DataFrame
X2 = X.copy()

# Categorical columns
cut_dummies = pd.get_dummies(X2["cut"], prefix="cut")
color_dummies = pd.get_dummies(X2["color"], prefix="color")
clarity_dummies = pd.get_dummies(X2["clarity"], prefix="clarity")

# Concatenate these variables
X2 = pd.concat([X2, cut_dummies, color_dummies, clarity_dummies], axis=1)

# Drop categorical variables
X2 = X2.drop(["cut", "color", "clarity"], axis=1)
X2

In [ ]:
# Create regressor object...doesn't work for categorical covariates
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)

# fit the regressor with x and y data
regressor.fit(X2, y)

In [ ]:
# Arrange for creating a range of values from min of 'x' to max of 'x' with a difference of 0.01
X_grid = np.arange(min(X2['carat']), max(X2['carat']), 0.01)

# Reshape for reshaping the data into a len(X_grid)*1 array, i.e. to make a column out of the X_grid value
X_grid = X_grid.reshape((len(X_grid), 1))

# Scatter plot for original data
plt.scatter(X2['carat'], y, s=20, edgecolor="black", c="darkorange", label="data")

# Plot predicted data
plt.scatter(X2['carat'], regressor.predict(X2), color="yellowgreen", label="randomforest", linewidth=1)
plt.title('Random Forest Regression')
plt.xlabel('Carat')
plt.ylabel('Price')
plt.show()

**Solving Overfitting Problem**

In [ ]:
# Split the data
X2_train, X2_test, y_train, y_test = train_test_split(X2, y, random_state=10)

In [ ]:
from sklearn import metrics

# Train a default RandomForestRegressor with random_state=10
model = RandomForestRegressor(random_state=10)
model.fit(X2_train, y_train)
print('Training MSE: ',
      metrics.mean_squared_error(y_train, model.predict(X2_train)))
print('Test MSE: ',
      metrics.mean_squared_error(y_test, model.predict(X2_test)))

In [ ]:
from sklearn import metrics

# Train a default RandomForestRegressor
model = RandomForestRegressor(n_estimators=50, max_depth=3, random_state=10)
  # The number of trees in the forest = 50 (default 100)
  # The maximum depth of the tree = 3 (default None)
model.fit(X2_train, y_train)
print('Training MSE: ',
      metrics.mean_squared_error(y_train, model.predict(X2_train)))
print('Test MSE: ',
      metrics.mean_squared_error(y_test, model.predict(X2_test)))

In [ ]:
from sklearn import metrics

# Train a default RandomForestRegressor
model = RandomForestRegressor(n_estimators=80, max_depth=5, random_state=10)
  # The number of trees in the forest = 80 (default 100)
  # The maximum depth of the tree = 5 (default None)
model.fit(X2_train, y_train)
print('Training MSE: ',
      metrics.mean_squared_error(y_train, model.predict(X2_train)))
print('Test MSE: ',
      metrics.mean_squared_error(y_test, model.predict(X2_test)))

In [ ]:
# Train your RandomForestRegressor
model = RandomForestRegressor(n_estimators=, max_depth=, random_state=10) # Put your `n_estimators` and `max_depth` values
model.fit(X2_train, y_train)
print('Training MSE: ',
      metrics.mean_squared_error(y_train, model.predict(X2_train)))
print('Test MSE: ',
      metrics.mean_squared_error(y_test, model.predict(X2_test)))

More options are described here: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

##### **Part 3. Gradient Boosting and XGBoost**

**3.1. Boosting**

**Background**
* Like bagging, boosting is a general approach that can be applied to many statistical learning methods for regression or classification. We only discuss boosting for decision trees.
* Recall that bagging involves creating multiple copies of the original training data set using the bootstrap, fitting a separate decision tree to each copy, and then combining all trees to create a single predictive model.
* Notably, each tree is built on a bootstrap data set, independent of the other trees.
* Boosting works in a similar way, except that the trees are grown *sequentially*: each tree is grown using information from previously grown trees.

**Bagging vs. Boosting**
>**Bagging:** It is a homogeneous weak learners' model that learns from each other independently in parallel and combines them for determining the model average.
>
>**Boosting:** It is also a homogeneous weak learners' model but works differently from bagging. In this model, learners learn sequentially and adaptively to improve model predictions of a learning algorithm.

<div>
<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/0*Fp5cX4lPib18xb1a.jpeg" width="800"/>
</div>

No. | Bagging | Boosting
--- | --- | ---
1. | The simplest way of combining predictions that belong to the same type. | A way of combining predictions that belong to the different types.
2. | Aim to decrease variance (overfitting), not bias. | Aim to decrease bias, not variance.
3. | Each model receives equal weight. | Models are weighted according to their performance.
4. | Each model is built independently. | New models are influenced by the performance of previously built models.
5. | Different training data subsets are selected using row sampling with replacement | Every new subset contains the elements that were misclassified
 | and random sampling methods from the entire training dataset. | by previous models.
6. | Classifiers are trained parallelly. | Classifiers are trained sequentially.


**Boosting Algorithm for Regression Trees**
1. Set $\hat{f}(x)=0$ and $r_i = y_i$ for all $i$ in the training set.
1. For $b = 1, 2, ..., B$, repeat:
>2.1. Fit a tree $\hat{f}^b$ with $d$ splits ($d+1$ terminal nodes) to the training data $(X, r)$.
>
>2.2. Update $\hat{f}$ by adding in a shrunken version of the new tree:
>
>$\hat{f}(x) \leftarrow \hat{f}(x) + \lambda \hat{f}^b(x)$.
>
>2.3. Update the residuals,
>
>$r_i \leftarrow r_i - \lambda \hat{f}^b(x)$.
>
1. Output the boosted model,
>
>$\hat{f}(x) = \Sigma_{b=1}^B \lambda \hat{f}^b(x)$.


**Idea behind this Procedure**
* Unlike fitting a single large decision tree to the data, which amounts to *fitting the data hard* and potentially overfitting, the boosting approach instead *learns slowly*.
* Given the current model, we fit a decision tree to the residuals from the model. We then add this new decision tree into the fitted function in order to update the residuals.
* Each of these trees can be rather small, with just a few terminal nodes, determined by the parameter $d$ in the algorithm.
* By fitting small trees to the residuals, we slowly improve $\hat{f}$ in areas where it does not perform well. The shrinkage parameter $\lambda$ slows the process down even further, allowing more and different shaped trees to attack the residuals.

**Hyperparameters for Boosting**
1. The *number of trees* $B$.
> Unlike bagging and random forests, boosting can overfit if $B$ is too large, although this overfitting tends to occur slowly if at all. We use cross-validation to select $B$.
1. The *shrinkage parameter* $\lambda$, a small positive number.
>This controls the rate at which boosting learns. Typical values are 0.01 or 0.001, and the right choice can depend on the problem. Very small $\lambda$ can require using a very large value of $B$ in order to achieve good performance.
1. The *number of splits* $d$ in each tree, which controls the complexity of the boosted ensemble.
>Often $d=1$ works well, in which case each tree is a *stump*, considering of a single split and resulting in an additive model. More generally, $d$ is the *interaction depth*, and controls the interaction order of the boosted model, since $d$ splits can involve at most $d$ variables.

**3.2. Gradient Boosting**

**Gradient Boosting Algorithm for Regression**

* **Input:** Data $\{ (x_i, y_i) \}_{i=1}^n$, and a differentiable **loss function** $L(y_i, F(x))$
>$i$: a data index, $n$: the number of observations
>
>*Loss function:* the function that a model wants to minimize, e.g., mean squared errors (MSE).

* **Step 1:** Initialize model with a constant value: $F_0(x) = argmin_\gamma \Sigma_{i=1}^n L(y_i, \gamma)$
>$y_i$: an observed value, $\gamma$: the predicted values
>
>For MSE, $F_0(x)$ is the average of $y_i$, and it becomes the initial leaf.

* **Step 2:** for $m=1$ to $M$:
>**(A)** Compute $r_{im}=-[\frac{\partial L(y_i, F(x_i))}{\partial F(x_i)}]_{F(x)=F_{m-1}(x)}$ for $i = 1, ..., n$
>* $r_{im}$ is a residual for sample $i$ and tree $m$.
>* $[\frac{\partial L(y_i, F(x_i))}{\partial F(x_i)}]$ is the derivative of the loss function or the ***gradient***.
>* $- \frac{d}{d Predicted} \frac{1}{2} (Observed - Predicted)^2 = (Observed - Predicted)$
>* Now we plug $F_{m-1}(x)$ in for $Predicted$.
>
>**(B)** Fit a regression tree to the $r_{im}$ values and create terminal regions $R_{jm}$, for $j = 1, ..., J_m$
>
>**(C)** For $j = 1, ..., J_m$, compute $\gamma_{jm}=argmin_{\gamma} \Sigma_{x_i \in R_{ij}} L(y_i, F_{m-1}(x_i) + \gamma)$
>* The output value for each leaf is the value for gamma that minimizes $\gamma_{jm}$ like what we did in **Step 1**.
>* Previous prediction $F_{m-1}$ is taken into account.
>* The summation considers sample in $R_{ij}$ only.
>
>**(D)** Update $F_m(x) = F_{m-1}(x) + \nu \Sigma_{j=1}^{J_m} \gamma_{jm} I(x \in R_{jm})$
>* Making a new prediction for each sample
>* We add up the output values $\gamma_{jm}$'s for all the leaves $R_{jm}$ where $x$ can be found.
>* $\nu \in [0, 1]$ is the learning rate that determines the effect each tree has on the final prediction, and this improves accuracy in the long run.

* **Step 3:** Output $F_M(x)$

**A. Gradient Boosting for Regression**

**Data Preparation**

In [ ]:
# Extract feature and target arrays
X, y = diamonds.drop('price', axis=1), diamonds[['price']]

In [ ]:
# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

In [ ]:
# Convert to pd.Categorical
for col in cats:
   X[col] = X[col].astype('category')

In [ ]:
# Check the variable types
X.dtypes

In [ ]:
# Create a copy of the original DataFrame
X2 = X.copy()

# Categorical columns
cut_dummies = pd.get_dummies(X2["cut"], prefix="cut")
color_dummies = pd.get_dummies(X2["color"], prefix="color")
clarity_dummies = pd.get_dummies(X2["clarity"], prefix="clarity")

# Concatenate these variables
X2 = pd.concat([X2, cut_dummies, color_dummies, clarity_dummies], axis=1)

# Drop categorical variables
X2 = X2.drop(["cut", "color", "clarity"], axis=1)
X2

**Model Implementation**

In [ ]:
# Import libraries and functions
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
# Define the evaluation procedure
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=10)
  # Number of folds = 5
  # Number of times cross-validator needs to be repeated = 10

In [ ]:
# Basic Gradinet Boosting
model = GradientBoostingRegressor()

# Estimate and evaluate the model
n_scores = cross_val_score(model, X2, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1) # The scikit-learn library makes the MAE negative so that it is maximized instead of minimized
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores))) # Larger negative MAE are better and a perfect model has a MAE of 0.

**B. Gradient Boosting for Classification**

**Data Preparation**

In [ ]:
# Extract feature and target arrays
X, y = diamonds.drop("cut", axis=1), diamonds[['cut']]

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Encode y to numeric
y_encoded = OrdinalEncoder().fit_transform(y)

In [ ]:
# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

In [ ]:
# Convert to pd.Categorical
for col in cats:
   X[col] = X[col].astype('category')

In [ ]:
# Create a copy of the original DataFrame
X2 = X.copy()

# Categorical columns
color_dummies = pd.get_dummies(X2["color"], prefix="color")
clarity_dummies = pd.get_dummies(X2["clarity"], prefix="clarity")

# Concatenate these variables
X2 = pd.concat([X2, color_dummies, clarity_dummies], axis=1)

# Drop categorical variables
X2 = X2.drop(["color", "clarity"], axis=1)
X2

**Model Implementation**

In [ ]:
# Import libraries and functions
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
# Define the model
model = GradientBoostingClassifier()

# Ddefine the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

In [ ]:
# Estimate and evaluate the model
n_scores = cross_val_score(model, X2, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

**3.3. XGBoost (eXtreme Gradient Boosting)**

**Basic Concepts**
* **Similarity Scores** $= \frac{\text{Sum of Residuals, Squared}}{\text{Number of Residuals} + \lambda}$
>*Sum of residuals, squared* is different from *sum of squared residuals*. Positive and negative residuals cancel out each other.
* **Regularization Parameter ($\lambda$)**
>It is intended to reduce the prediction's sensitivity to individual observations.
>When $\lambda > 0$, the **similarity scores** and **gain** are smaller (or easier to **prune** leaves).
* **Gain** $= \text{Similarity}_\text{Left} + \text{Similarity}_\text{Right} - \text{Similarity}_\text{Root}$
>An improvement in similarity scores after splitting the residuals.
* **Pruning**
>If the **gain** of splitting the residuals into two groups is greater (smaller) than $\gamma$, retain (remove) this branch.
* **Output Value**  $= \frac{\text{Sum of Residuals}}{\text{Number of Residuals} + \lambda}$
> $\lambda$ reduces the prediction's sensitivity to an individual observation. Note that the output value for a leaf is simply the average of the residuals in the leaf when $\lambda=0$.
* **Learning Rate ($\eta$)**
>XGBoost makes new predictions by stating with the initial prediction just like other Gradient Boost. And it adds the output of the tree scaled by a **learning rate**. The default value is 0.3.

**XGBoost vs Gradient Boosting**
* XGBoost is a more regularized form of Gradient Boosting. XGBoost uses advanced regularization (L1 & L2), which improves model generalization capabilities.
* XGBoost delivers high performance as compared to basic Gradient Boosting. Its training is very fast and can be parallelized across clusters.

**When to use XGBoost?**
* When there is a larger number of training samples. Ideally, greater than 1000 training samples and less 100 features or we can say when the number of features < number of training samples.
* When there is a mixture of categorical and numeric features or just numeric features.

**When not to use XGBoost?**
* Image Recognition
* Computer Vision
* When the number of training samples is significantly smaller than the number of features.

In [ ]:
# Import XGBoost
import xgboost as xgb # Already installed in Colab!

**Data Preparation**

In [ ]:
# Extract feature and target arrays
X, y = diamonds.drop('price', axis=1), diamonds[['price']]

In [ ]:
# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

In [ ]:
# Convert to Pandas category
for col in cats:
   X[col] = X[col].astype('category')

In [ ]:
# Check the variable types
X.dtypes

In [ ]:
# Create a copy of the original DataFrame
X2 = X.copy()

# Categorical columns
color_dummies = pd.get_dummies(X2["color"], prefix="color")
clarity_dummies = pd.get_dummies(X2["clarity"], prefix="clarity")

# Concatenate these variables
X2 = pd.concat([X2, color_dummies, clarity_dummies], axis=1)

# Drop categorical variables
X2 = X2.drop(["cut", "color", "clarity"], axis=1)
X2

In [ ]:
# Check the variable types
X2.dtypes

In [ ]:
# Split the data
X2_train, X2_test, y_train, y_test = train_test_split(X2, y, random_state=1)

**Model Implementation**

In [ ]:
# Instantiate an XGBoost regressor
# Objective is reg:squarederror for regression tasks
reg = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, seed=42)

In [ ]:
# Train the regressor
reg.fit(X2_train, y_train)

In [ ]:
from sklearn.metrics import mean_absolute_error

# Predict values
preds = reg.predict(X2_test)

# Define and print MSE
mae = mean_absolute_error(y_test, preds)
print(f"MAE of the base model: {mae:.3f}")